In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from utils import create_param_list

In [3]:
nconf = 31
ndata = 200
L = 64
T_cr = 2.269 # critical temp for 2d ising
t_start = 2.1

prm_list, t_end = create_param_list(nconf=nconf, t_start=t_start, L=L, model_name="2d_Ising")
# print(prm_list)
print(f"データ数:{nconf*ndata}")

データ数:6200


In [14]:
img_list = []
for num in range(nconf):
    for iconf in range(ndata):
        file = f"{prm_list[num][1]}{iconf}.npy"
        img_list.append(np.load(file))

img_dataset = np.array(img_list)

In [17]:
# flatten+kmeans
n_clusters = 2

img_dataset = img_dataset.reshape(img_dataset.shape[0], -1)
model = KMeans(n_clusters=n_clusters, n_init='auto').fit(img_dataset)

(1550, 64, 64)
(1550, 4096)


In [ ]:
# pca+kmeans

In [ ]:
# CNN+kmeans